$$
Белорусский\;государственный\; университет
$$
$$
Механико-математический\;факультет
$$
$$
Кафедра\;дифференциальных\;уравнений\; и\; системного\; анализа
$$
$ $

$$
 \Large\bf Математические\; основы\; защиты\; информации
$$

# Тема 5. Факторизация чисел. Экспоненциальные методы.

$ $

доцент Чергинец Дмитрий Николаевич

## Разложение числа на множители
**Основная теорема арифметики.**  
Каждое натуральное число $\;n > 1\;$ единственным образом представимо в виде
$$
  n=p_1^{\alpha_1}*p_2^{\alpha_2}*\dots*p_k^{\alpha_k},
$$
где $\;\alpha_i\in\mathbb{N},\;$ $\;p_1<p_2<\dots<p_k\;$ - простые числа.

In [1]:
from sympy.ntheory import factorint
factorint(621)

{3: 3, 23: 1}

# 1. Метод пробных делений
Input: $\;n$  -- составное.   

  
Output: factors -- словарь, ключами которого являются простые делители $\;p_i\;$ числа $\;n,\;$ а значениями ключей `factors[pi]` - степени $\alpha_i,$
$$
  n=p_1^{\alpha_1}*p_2^{\alpha_2}*\dots*p_k^{\alpha_k}.
$$

In [2]:
def Trial(n):
    factors = {}
    d = 2
    while d < n**(1/2):
        if n%d == 0:
            n = n//d
            factors[d] = 1
            while n%d == 0:
                n = n//d
                factors[d] += 1
        d += 1
    if n>1:
        factors[n] = 1
    return factors
Trial(621)

{3: 3, 23: 1}

## Сложность алгоритма пробных делений
Для количества арифметических операций справедлива оценка
$$f(n)\geq n^{1/2}-1.$$
Для числа $\;n\;$ и количества его бит $ \left<n\right>$ справедливы соотношения
$$
    \left<n\right>=[\log_2n]+1\;\;\Rightarrow\;\; n=O(2^{\left<n\right>}).
  $$
Поэтому
 $$
    T(N):=\max\limits_{\left<n\right>\leq N} f(n)\geq O\left(2^{\frac{N}{2}}\right).
  $$ 
Метод пробных делений - экспоненциальный алгоритм.

## <font color='red'>Задание 1.</font>
Улучшить код функции `Trial`. Для этого можно, например, применить функции `isprime` или `primerange` из `sympy.ntheory`. Сравнить фактическую скорость вычислений полученной Вами функции со скоростью работы функции `Trial`.

Изначальный код функции можно услучшить, если использовать генераторный объект.

Улучшение функции `Trial` при помощи функции `isprime`.

In [3]:
from sympy.ntheory import isprime

def Trial_isprime(n: int)->dict:
    factors = {}
    
    if n%2 == 0:
        n //= 2
        factors[2] = 1
        while n%2 == 0:
            n //= 2
            factors[2] += 1
    d = 3
    while d < n**(1/2):
        if isprime(d):
            if n%d == 0:
                n = n//d
                factors[d] = 1
                while n%d == 0:
                    n = n//d
                    factors[d] += 1
        d += 2
    if n>1:
        factors[n] = 1
    return factors

Улучшение функции `Trial` при помощи функции `primerange`.

In [4]:
from sympy.ntheory import primerange

def Trial_primerange(n: int) -> dict:
    factors = dict()
    for d in primerange(n**(1/2)):
        if n%d == 0:
            n = n//d
            factors[d] = 1
            while n%d == 0:
                n = n//d
                factors[d] += 1
    if n>1:
        factors[n] = 1
    return factors

Сравним фактические скорости вычислений улучшенных функций и изначальной.

In [5]:
import time, random

numbers = [random.randint(4,100_000) for _ in range(100_000)]

start = time.time()
for element in numbers:
    Trial(element)
    
stop = time.time()
print('Time Trial: ', stop-start)

start = time.time()
for element in numbers:
    Trial_isprime(element)
    
stop = time.time()
print('Time Trial_isprime: ', stop-start)

start = time.time()
for element in numbers:
    Trial_primerange(element)
    
stop = time.time()
print('Time Trial_primerange: ', stop-start)

Time Trial:  4.095377683639526
Time Trial_isprime:  7.416076421737671
Time Trial_primerange:  36.609939098358154


# 2.  $\;\bf\rho$-метод Полларда
**Парадокс дней рождения**  
- Вероятность того, что среди 23 человек есть хотя бы 2 человека с одинаковым днем рождения, больше $\frac12.$ 

- Вероятность того, что Ваш день рождения совпадает с днем рождения хотя бы одного человека из группы, состоящей из 158 человек, меньше $\frac12.$ 

- Вероятность того, что среди 58 человек есть хотя бы 2 человека с одинаковым днем рождения, больше $\;0.99.$ 


Парадокс здесь в том, что наше предположение о данной вероятности скорее всего значительно не совпадает с реальной вероятностью, вычисленной при помощи формул теории вероятностей.


## Идея $\bf\rho$-метода Полларда

Пусть $\;p\;$ -- делитель числа $\;n,\;$   нам даны случайные $\;l+1\;$ число
$$
  x_0,x_1,\dots,x_l\in\mathbb{Z}_n.
$$

С точки зрения парадокса дней рождения с большой вероятностью найдутся такие $\;i,j,\;$ $\;0\leq j<i\leq l,\;$ что
$$
  x_i\equiv x_j  \mod p.
$$
Тогда 
$$
\;d:=\gcd(x_i-x_j,n)\geq p,\;
$$
если к тому же $\;d<n,\;$ то нам повезло и мы нашли делитель.


## Алгоритм Черепаха
 **Алгоритм Черепаха с генератором случайных чисел**  
Input:  $\;n\in\mathbb{N}$ -- составное число.
  
Output: $\;d\;$ - нетривиальный делитель числа $n.$
  
1$.$ Задаем начальные данные: $\;i:=0,\;$ выбираем случайное $\;x_0\;$ из кольца $\;\mathbb{Z}_n.$
  
2$.$ Присваиваем $\;i:=i+1\;$ и выбираем случайное $\;x_i\;$ из кольца $\;\mathbb{Z}_n.$
  
3$.$ Для $\;j=0,1,\dots, i-1\;$ выполняем шаги 3.1, 3.2:
  
  $\qquad$ 3.1. Вычисляем $\;d:=\gcd(x_i-x_j,n).$
  
  $\qquad$ 3.2. Если $\;1<d<n,\;$ то выдаем результат $\;d,\;$ конец алгоритма.
  
4$.$ Переходим к шагу $2.$
  

In [6]:
from random import randrange
from math import gcd
def TortoiseRand(n):
    i = 0
    x = [randrange(n)]
    while 1:
        i += 1
        x.append(randrange(n))
        for j in range(i):
            d = gcd(x[i]-x[j], n)
            if 1 < d < n:
                return d

TortoiseRand(11*13)        

13

### Сравнение времени работы алгоритма Черепаха с Методом пробных делений

In [7]:
from sympy import randprime

b = 16

start_time = time.time()
for _ in range(100):
    p1 = randprime(1<<b-1, 1<<b)
    p2 = randprime(1<<b-1, 1<<b)
    n = p1*p2
    Trial(n)    
print("Trial:--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
for _ in range(100):
    p1 = randprime(1<<b-1, 1<<b)
    p2 = randprime(1<<b-1, 1<<b)
    n = p1*p2
    TortoiseRand(n)    
print("TortoiseRand:--- %s seconds ---" % (time.time() - start_time))

Trial:--- 3.6850345134735107 seconds ---
TortoiseRand:--- 2.056764841079712 seconds ---


## Анализ алгоритма Черепаха
Данный алгоритм можно представить себе как игровое поле, составленное из упорядоченных клеток 
$$
x_0,\; x_1,\; x_2,\; x_3,\; x_4,\; x_5,\;\dots.
$$ 

Клетки $\;x_i\;$ и $\;x_j\;$  называются "Одинаковыми", если
	 $$\gcd(x_i-x_j,n)>1.$$
По данным клеткам начиная с $\;x_0\;$ по порядку двигается Черепаха. Задача Черепахи найти одинаковые фишки, для которых к тому же 
$$\;\gcd(x_i-x_j,n)<n.\;
$$

 Данный алгоритм является вероятностным, но всегда выдает правильный ответ.
 
 
 В алгоритме Черепаха третий шаг является очень трудоемким, так как на нем $\;i\;$ раз вычисляется НОД.     Попробуем уменьшить количество этих операций

## Генерация случайной последовательности
Вместо генерации последовательности функцией `randrange` будем генерировать последовательность специальным образом:

1. $x_0$ выбираем случайным. 

2. Остальные $\;x_i\;$ вычисляем при помощи рекуррентной формулы
 $$
   x_i:=f(x_{i-1}),
 $$
где  
$$
   f:\mathbb{Z}_n\rightarrow\mathbb{Z}_n.
 $$

Как правило в качестве функции $\;f\;$ берут многочлен второй степени
 $$
   f(x):=x^2+1\mod n\qquad\text{или}\qquad f(x):=x^2+a\mod n.
 $$

Пара $\;(f,x_0)\;$ однозначно задает последовательность 
$$
x_0,\; x_1,\; x_2,\; x_3,\; x_4,\; x_5,\;\dots.
$$ 

### Алгоритм Черепаха с генерацией последовательности при помощи многочлена

In [8]:
def Tortoise_fSeq(n, a, x0):
    f = lambda x: (x*x + a) % n
    i = 0
    x = [x0]
    while 1:
        i += 1
        x.append(f(x[i-1]))
        for j in range(i):
            d = gcd(x[i]-x[j], n)
            if 1 < d:
                if d < n:
                    return d
                else:
                    return 'Unsuccessful Sequence'  # неудачная последовательность
Tortoise_fSeq(11*13, 1, 1)  

13

In [9]:
b = 16

start_time = time.time()
for _ in range(100):
    p1 = randprime(1<<b-1, 1<<b)
    p2 = randprime(1<<b-1, 1<<b)
    n = p1*p2
    Tortoise_fSeq(n, 1, 1)    
print("Tortoise_fSeq:--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
for _ in range(100):
    p1 = randprime(1<<b-1, 1<<b)
    p2 = randprime(1<<b-1, 1<<b)
    n = p1*p2
    TortoiseRand(n)    
print("TortoiseRand:--- %s seconds ---" % (time.time() - start_time))

Tortoise_fSeq:--- 1.985710859298706 seconds ---
TortoiseRand:--- 1.9072694778442383 seconds ---


## Определение длины хвоста $\;\mu\;$ и периода $\;\lambda$
Пусть $\;\mu,\lambda,\;$ $\;\lambda>0,\;$ это наименьшие числа среди тех, для которых справедливо сравнение
 $$
   x_\mu\equiv x_{\mu+\lambda}\mod p\qquad \Leftrightarrow\qquad\text{НОД}(x_\mu-x_{\mu+\lambda},n)>1.
 $$


![blockchain](images/rho.png)



## Цикличность

$$
   x_\mu\equiv x_{\mu+\lambda}\mod p\quad\Rightarrow \quad f(x_{\mu})\equiv f(x_{\mu+\lambda})\mod p.
 $$
 
 
 Тогда для всех $\;m\in\mathbb{N},\;$ подействовав на обе части сравнения $\;m\;$ раз функцией $\;f,\;$ получаем сравнение
 
 $$
f^m(x_{\mu})\equiv f^m(x_{\mu+\lambda})\mod p\quad\Rightarrow \quad   x_{\mu+m}\equiv x_{\mu+\lambda+m}\mod p.
 $$


![blockchain](images/rho.png)


 


## Алгоритм Черепаха и Заяц
Input:  $\;n\in\mathbb{N}$ -- составное число.  
  $\qquad a,x_0\in\mathbb{Z}_n\;$ задают последовательность: $\;x_i=f(x_{i-1}),\;$
$\;f(x):=x^2+a\mod n.$   
Output: $\;d\;$ - нетривиальный делитель числа $\;n.$

 
1. Задаем начальные позиции Черепахи и Зайца  
     $\qquad Tortoise:=x_0,\qquad Hare:=x_0.$
  
2. Черепаха "переползает" на следующий элемент, а Заяц "прыгает" через элемент  
     $\qquad Tortoise:=f(Tortoise),\quad Hare:=f(f(Hare)).$
3. Вычисляем $\;d:=\gcd(Tortoise-Hare,n).$
  
4. Если $\;1<d<n,\;$ то выдаем результат $\;d,\;$ конец алгоритма.

5. Если $\;d=n,\;$  то нет решения,  конец алгоритма.
  
6.  Переходим к шагу $2.$  


In [10]:
def TortoiseHare(n, a, x0):
    f = lambda x: (x*x + a) % n
    Tortoise = x0
    Hare = x0
    while 1:
        Tortoise = f(Tortoise)
        Hare = f(f(Hare))
        d = gcd(Hare - Tortoise, n)
        if 1 < d:
            if d < n:
                return d
            else:
                return 'Unsuccessful Sequence'
print(TortoiseHare(11*13, 1, 1))
print(TortoiseHare(11*13, 2, 1))

Unsuccessful Sequence
13


### Сравнение алгоритма Черепаха с алгоритмом Черепаха и Заяц 

In [11]:
b = 16

start_time = time.time()
for _ in range(100):
    p1 = randprime(1<<b-1, 1<<b)
    p2 = randprime(1<<b-1, 1<<b)
    n = p1*p2
    Tortoise_fSeq(n, 1, 1)    
print("Tortoise_fSeq: --- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
for _ in range(100):
    p1 = randprime(1<<b-1, 1<<b)
    p2 = randprime(1<<b-1, 1<<b)
    n = p1*p2
    TortoiseHare(n, 1, 1)    
print("TortoiseHare: --- %s seconds ---" % (time.time() - start_time))

Tortoise_fSeq: --- 1.7777125835418701 seconds ---
TortoiseHare: --- 0.05643320083618164 seconds ---


## Количество вычислений НОД в алгоритме Черепаха
Во время $\;i$-ой итерации цикла `while` алгоритм вычисляет $\;i\;$ наибольших общих делителя. Согласно нашим обозначениям:

$\;\mu,\lambda,\;$ $\;\lambda>0,\;$ это наименьшие числа среди тех, для которых справедливо сравнение
 $$
   x_\mu\equiv x_{\mu+\lambda}\mod p\qquad \Leftrightarrow\qquad\text{НОД}(x_\mu-x_{\mu+\lambda},n)>1.
 $$
Поэтому на итерации $\;i = \mu+\lambda\;$ алгоритм закончит работу.

Количество НОД в алгоритме Черепаха:
  $$
    1+2+3+\dots+(\mu+\lambda-1)+(\mu+1)=
  $$
  $$
    =\frac12(\mu+\lambda-1)(\mu+\lambda)+(\mu+1)\leq \frac12(\mu+\lambda+1)(\mu+\lambda).
  $$  

## Количество вычислений НОД в алгоритме Черепаха и Заяц
Количество вычислений НОД в алгоритме Черепаха и Заяц будет совпадать с индексом $\;i\;$ конечной позиции Черепахи $\;x_i.\;$  

Для нахождения конечной позиции Черепахи необходимо найти минимальное $\;i\in\mathbb{N},\;$ удовлетворяющее условиям
 $$
   \left\{\begin{array}{l}i\geq\mu,\\
 i\equiv0\mod\lambda,\\
 i\rightarrow \min.
 \end{array}\right.\qquad\Rightarrow\qquad i=\lambda\lceil\frac{\mu}{\lambda}\rceil.
 $$
 
 Для $\;i\; $ справедлива оценка
 $$
 \mu\leq\lambda\lceil\frac{\mu}{\lambda}\rceil<\mu+\lambda.
 $$

## <font color='red'>Задание 2.</font>
В алгоритме Черепаха и Заяц последовательность $\;x_i\;$ не хранится, из-за этого приходится вычислять позицию Черепахи при помощи функции $\;f\;.$ Реализовать алгоритм Черепаха и Заяц с хранением последовательности $\;x_i.\;$ Сравнить скорость полученной функции с функцией `TortoiseHare`. В какой из функций выполняется больше арифметических операций? 

In [12]:
def TortoiseHareSequence(n, a, x0):
    f = lambda x: (x*x + a) % n
    i = 0
    x = {i:x0}
    while 1:
        i += 2
        x[i - 1] = f(x[i - 2])
        x[i] = f(x[i - 1])
        d = gcd(x[i] - x[i//2], n)
        if 1 < d:
            if d < n:
                return d
            else:
                return 'Unsuccessful Sequence'           
print(TortoiseHareSequence(11*13, 1, 1))
print(TortoiseHareSequence(11*13, 2, 1))

Unsuccessful Sequence
13


Сравним скорость работы функции `TortoiseHareSequence` с функцией `TortoiseHare`.

In [36]:
b = 16

start_time = time.time()
for _ in range(100):
    p1 = randprime(1<<b-1, 1<<b)
    p2 = randprime(1<<b-1, 1<<b)
    n = p1*p2
    TortoiseHare(n, 1, 1)    
print("TortoiseHare: --- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
for _ in range(100):
    p1 = randprime(1<<b-1, 1<<b)
    p2 = randprime(1<<b-1, 1<<b)
    n = p1*p2
    TortoiseHareSequence(n, 1, 1)    
print("TortoiseHareSequence: --- %s seconds ---" % (time.time() - start_time))

TortoiseHare: --- 0.05776786804199219 seconds ---
TortoiseHareSequence: --- 0.058988332748413086 seconds ---


Основное различие в количестве арифметических операций наблюдается в теле цикла `while`: в функции `TortoiseHareSequence` совершается 12 операций за одну итерацию, а в функции `TortoiseHare` — 9.

Однако при этом из-за того, что в функции `TortoiseHareSequence` 2 раза вычисляется лямбда-функция `f` за одну итерацию и элемент $x_{i}$ хранится в словаре, можем получить небольшой выигрыш в скорости, поскольку использование словарей является достаточно эффективным.

## Черепаха и Ахиллес 
Рассмотрим еще один вариант $\rho$-метода Полларда. Черепаха как обычно будет ползти по всем элементам $\;x_i\;$ последовательности, а
вместо зайца по элементам  
$$
  \bf x_{2^0},x_{2^1},\dots,x_{2^k},\dots
$$
будет двигаться Ахиллес
$$
   {\bf x_{2^0}}, \;{\bf x_{2^1}},x_3,\;{\bf x_{2^2}},x_5,x_6,x_7,\;{\bf x_{2^3}},\; x_9, x_{10}, x_{11},x_{12}, x_{13}, x_{14}, x_{15},\; {\bf x_{2^4}},\;x_{17},\dots
$$




Причем, как утверждал Зенон Элейский, Ахиллес никогда не догонит черепаху.


## Алгоритм Черепаха и Ахиллес
Input:  $\;n\in\mathbb{N}$ -- составное число.  
  $\qquad a,x_1\in\mathbb{Z}_n\;$ задают последовательность: $\;x_i=f(x_{i-1}),\;$
$\;f(x):=x^2+a\mod n.$   
Output: $\;d\;$ - нетривиальный делитель числа $\;n.$

 
1$.$ Задаем начальные позиции Черепахи и Ахиллеса  
     $\qquad Tortoise:=x_1,\qquad Achilles:=x_1, \qquad k=0.$
  
2$.$ Для $\;i=1,2,3,\dots,2^{k}\;$ выполняем шаги 2.1-2.4:
    
    
$\qquad$ 2.1.  Передвигаем Черепаху  $\;Tortoise:=f(Tortoise).$

$\qquad$ 2.2.   Вычисляем  $\;d:=\gcd(Tortoise-Achilles,n).$
    
$\qquad$ 2.3.  Если $\;1<d<n,\;$ то выдаем результат $\;d,\;$ конец алгоритма.

$\qquad$ 2.4. Если $\;d=n,\;$  то нет решения,  конец алгоритма.
    

3$.$  Передвигаем Ахиллеса с $\;x_{2^{k}}\;$ на элемент $\;x_{2^{k+1}}\;$
    $$
      Achilles:=Tortoise,\qquad k:=k+1,
    $$
$\;\;$ переходим на шаг $2.$      



## Количество  НОД в алгоритме Черепаха и Ахиллес
Посчитаем сколько раз данный алгоритм вычисляет НОД. Наибольший общий делитель будет больше единицы когда Ахиллес будет находиться на элементе $\;x_{2^{k}},\;$ а Черепаха на $\;x_{2^{k}+\lambda}\;,$ где $\;k\;$ -- наименьшее целое, удовлетворяющее неравенствам
  $$
\left\{\begin{array}{l}
\mu\leq 2^{k},\\
\lambda\leq 2^{k},\\
k\rightarrow\min.
\end{array}      
\right.
\qquad\Rightarrow\qquad\left\{\begin{array}{l}
\log_2\mu\leq k,\\
\log_2\lambda\leq k,\\
k\rightarrow\min.
\end{array}      
\right.\qquad\Rightarrow\qquad
k:=\max\{\lceil\log_2\mu\rceil,\lceil\log_2\lambda\rceil\},
$$

Алгоритм Черепаха и Ахиллес вычисляет
НОД следующее количество раз
$$
2^{k}+\lambda-1=\lambda-1+2^{\max\{\lceil\log_2\mu\rceil,\lceil\log_2\lambda\rceil\}}.
$$


Справедлива следующая оценка
$$    
  \lambda+\max\{\mu,\lambda\}\leq\lambda+2^{\max\{\lceil\log_2\mu\rceil,\lceil\log_2\lambda\rceil\}}.
  $$ 
  Очевидно, что
  $$
    \lambda\lceil\frac{\mu}{\lambda}\rceil<\mu+\lambda\leq \lambda+\max\{\mu,\lambda\}\leq
    \lambda+2^{\max\{\lceil\log_2\mu\rceil,\lceil\log_2\lambda\rceil\}}.
    $$
   Поэтому количество вычислений НОД в алгоритме Черепаха и Заяц не превосходит количества вычислений НОД в алгоритме Ахиллес и Черепаха. 

## Сравнение алгоритмов по количеству вычилений $f$
В алгоритме Черепаха и Заяц функция $\;f\;$ вычисляется $\;3i\;$ раз
$$3\mu\leq 3i=3 \lambda\lceil\frac{\mu}{\lambda}\rceil$$
В алгоритме Ахиллес и Черепаха 
   $$\lambda+2^{\max\{\lceil\log_2\mu\rceil,\lceil\log_2\lambda\rceil\}}<\lambda+\max\{2\mu,2\lambda\}.$$

Поэтому количество вычислений функции $\;f\;$ в алгоритме Ахиллес и Черепаха  не превосходит количества вычислений $\;f\;$ в алгоритме Черепаха и Заяц.

Тем не менее, при вычислении функции $\;f\;$ выполняется 3 арифметических операции, а при вычислении НОД: $\;O(\log n).$   

## Парадокс дней рождения
**Теорема.**

Пусть $X$ -- множество из $\;p\;$ элементов.
    
$\beta\in\mathbb{R}\;$ удовлетворяет неравенствам
     
$$\beta>0,\qquad{l:=1+[\sqrt{2\beta p}]<p.}
      $$

Тогда вероятность того, что среди $\;l+1\;$ случайным образом взятого элемента
$$
      x_0,x_1,\dots,x_{l}\in X
$$
любые два попарно различны, равна
$$
      (1-1/p)(1-2/p)\dots(1-l/p).
$$
Причем справедливо неравенство
$$
      (1-1/p)(1-2/p)\dots(1-l/p)<e^{-\beta}.
$$


## Вычислительная сложность $\bf\rho$-метода Полларда
Пусть $\;p\;$ - наименьший нетривиальный делитель числа $\;n.$

Найдем $\;l\in\mathbb{N},\;$ при котором среди элементов 
    $$
      x_0,x_1,\dots,x_{l}\in\mathbb{Z}_p 
    $$
с вероятностью, не меньшей $\;1/2,\;$ есть два одинаковых элемента
$$
  x_\mu\equiv x_{\mu+\lambda}\mod p.
$$

Согласно предыдущей теореме
$$  
 e^{-\beta}=1/2\qquad\Rightarrow\qquad \beta=\ln 2\qquad\Rightarrow\qquad
  l=1+[\sqrt{2p\ln 2}]=O(p^{1/2}).
$$  

На вычисление одного НОД уходит $\;O(\ln n)\;$ арифметических операций, поэтому 
$O(p^{1/2}\ln n)$ - среднее количество арифметических операций $\rho$-алгоритма Полларда.


## <font color='red'>Задание 3.</font>
Реализовать алгоритм Черепаха и Ахиллес. Сравнить скорость полученной функции с функцией `TortoiseHare`. В какой из функций выполняется больше арифметических операций? 

In [14]:
def TortoiseAchilles(n, a, x1):
    f = lambda x: (x*x + a) % n
    Tortoise = x1
    Achilles = x1
    k = 0
    while 1:
        for _ in range(2**k):
            Tortoise = f(Tortoise)
            d = gcd(Tortoise - Achilles, n)
            if 1 < d < n:
                return d
            if d == n:
                return 'Unsuccessful Sequence'
        Achilles = Tortoise
        k += 1
print(TortoiseAchilles(11*13, 1, 1))
print(TortoiseAchilles(11*13, 2, 1))

11
13


Сравним скорость работы функции `TortoiseAchilles` с функцией `TortoiseHare`.

In [15]:
b = 16

start_time = time.time()
for _ in range(100):
    p1 = randprime(1<<b-1, 1<<b)
    p2 = randprime(1<<b-1, 1<<b)
    n = p1*p2
    TortoiseHare(n, 1, 1)    
print("TortoiseHare: --- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
for _ in range(100):
    p1 = randprime(1<<b-1, 1<<b)
    p2 = randprime(1<<b-1, 1<<b)
    n = p1*p2
    TortoiseAchilles(n, 1, 1)    
print("TortoiseAchilles: --- %s seconds ---" % (time.time() - start_time))

TortoiseHare: --- 0.05421853065490723 seconds ---
TortoiseAchilles: --- 0.05194234848022461 seconds ---


Из вышеприведенной информации о сравнении алгоритмов, в частности из предложений:
1. количество вычислений НОД в алгоритме Черепаха и Заяц не превосходит количества вычислений НОД в алгоритме Ахиллес и Черепаха,
2. количество вычислений функции $f$ в алгоритме Ахиллес и Черепаха  не превосходит количества вычислений $f$ в алгоритме Черепаха и Заяц

следует, что для того, чтобы сказать, в какой из функций выполняется больше арифметических операций, необходимо подсчитать количество вычислений НОД и функции $f$, иными словами — количество итераций.

Функция `TortoiseHare` совершает $i = \lambda\lceil\frac{\mu}{\lambda}\rceil$ интераций.

Функция `TortoiseAchilles` вычисляет НОД (а значит и итерируется) $2^{k}+\lambda-1=\lambda-1+2^{\max\{\lceil\log_2\mu\rceil,\lceil\log_2\lambda\rceil\}}$ раз.

Воспользуемся оценкой $\lambda\lceil\frac{\mu}{\lambda}\rceil<\mu+\lambda$, 

тогда в предположении, что $\max\{\lceil\log_2\mu\rceil,\lceil\log_2\lambda\rceil\} = \lceil\log_2\mu\rceil$, получим, что

$\lambda-1+2^{\max\{\lceil\log_2\mu\rceil,\lceil\log_2\lambda\rceil\}} = \lambda-1+\lceil\log_2\mu\rceil \leq \lambda+\log_2\mu < \mu+\lambda$.

Как можно видеть, количество итераций и в одном алгоритме, и в другом совпадают, однако для вывода стоит обратить внимание на предложение 1. Согласно этому предложению в функции `TortoiseHare` будет меньше (меньше или равно) операций, чем в функции `TortoiseAchilles`.

## <font color='red'>Задание 4.</font>
Пусть 
$$
  f(x)=x^2+1,\qquad x_0=1.
$$
Для чисел $\;n_1\;$  и  $\;n_2\;$  выяснить, существуют ли такие $\;i\;$ и $\;j,\;$ что  
$$
1<gcd(x_i-x_j,n)<n.
$$

In [37]:
def TortoiseHare_not_n(n, a, x0):
    f = lambda x: x*x + 1
    Tortoise = x0
    Hare = x0
    while 1:
        Tortoise = f(Tortoise)
        Hare = f(f(Hare))
        d = gcd(Hare - Tortoise, n)
        if 1 < d:
            if d < n:
                return d
            else:
                return 'Unsuccessful Sequence'

In [38]:
n1 = 41053
print(TortoiseHare_not_n(n1, 1, 1))
n2 = 52357
print(TortoiseHare_not_n(n2, 1, 1))

Unsuccessful Sequence
Unsuccessful Sequence


попробовать с помош\щью Черепаха и Ахиллес

Функция `pollard_rho(n, s=2, a=1, retries=5, seed=1234, max_steps=None, F=None)` из модуля `sympy.ntheory.residue_ntheory` имеет следующий вид

In [18]:
import random
from sympy.core.numbers import igcd

def pollard_rho(n, s=2, a=1, retries=5, seed=1234, max_steps=None, F=None):
    r"""
    Use Pollard's rho method to try to extract a nontrivial factor
    of ``n``. The returned factor may be a composite number. If no
    factor is found, ``None`` is returned.

    The algorithm generates pseudo-random values of x with a generator
    function, replacing x with F(x). If F is not supplied then the
    function x**2 + ``a`` is used. The first value supplied to F(x) is ``s``.
    Upon failure (if ``retries`` is > 0) a new ``a`` and ``s`` will be
    supplied; the ``a`` will be ignored if F was supplied.

    The sequence of numbers generated by such functions generally have a
    a lead-up to some number and then loop around back to that number and
    begin to repeat the sequence, e.g. 1, 2, 3, 4, 5, 3, 4, 5 -- this leader
    and loop look a bit like the Greek letter rho, and thus the name, 'rho'.

    For a given function, very different leader-loop values can be obtained
    so it is a good idea to allow for retries:

    >>> from sympy.ntheory.generate import cycle_length
    >>> n = 16843009
    >>> F = lambda x:(2048*pow(x, 2, n) + 32767) % n
    >>> for s in range(5):
    ...     print('loop length = %4i; leader length = %3i' % next(cycle_length(F, s)))
    ...
    loop length = 2489; leader length =  42
    loop length =   78; leader length = 120
    loop length = 1482; leader length =  99
    loop length = 1482; leader length = 285
    loop length = 1482; leader length = 100

    Here is an explicit example where there is a two element leadup to
    a sequence of 3 numbers (11, 14, 4) that then repeat:

    >>> x=2
    >>> for i in range(9):
    ...     x=(x**2+12)%17
    ...     print(x)
    ...
    16
    13
    11
    14
    4
    11
    14
    4
    11
    >>> next(cycle_length(lambda x: (x**2+12)%17, 2))
    (3, 2)
    >>> list(cycle_length(lambda x: (x**2+12)%17, 2, values=True))
    [16, 13, 11, 14, 4]

    Instead of checking the differences of all generated values for a gcd
    with n, only the kth and 2*kth numbers are checked, e.g. 1st and 2nd,
    2nd and 4th, 3rd and 6th until it has been detected that the loop has been
    traversed. Loops may be many thousands of steps long before rho finds a
    factor or reports failure. If ``max_steps`` is specified, the iteration
    is cancelled with a failure after the specified number of steps.

    Examples
    ========

    >>> from sympy import pollard_rho
    >>> n=16843009
    >>> F=lambda x:(2048*pow(x,2,n) + 32767) % n
    >>> pollard_rho(n, F=F)
    257

    Use the default setting with a bad value of ``a`` and no retries:

    >>> pollard_rho(n, a=n-2, retries=0)

    If retries is > 0 then perhaps the problem will correct itself when
    new values are generated for a:

    >>> pollard_rho(n, a=n-2, retries=1)
    257

    References
    ==========

    .. [1] Richard Crandall & Carl Pomerance (2005), "Prime Numbers:
           A Computational Perspective", Springer, 2nd edition, 229-231

    """
    n = int(n)
    if n < 5:
        raise ValueError('pollard_rho should receive n > 4')
    prng = random.Random(seed + retries)
    V = s
    for i in range(retries + 1):
        U = V
        if not F:
            F = lambda x: (pow(x, 2, n) + a) % n
        j = 0
        while 1:
            if max_steps and (j > max_steps):
                break
            j += 1
            U = F(U)
            V = F(F(V))  # V is 2x further along than U
            g = igcd(U - V, n)
            if g == 1:
                continue
            if g == n:
                break
            return int(g)
        V = prng.randint(0, n - 1)
        a = prng.randint(1, n - 3)  # for x**2 + a, a%n should not be 0 or -2
        F = None
    return None

pollard_rho(121)

11

## <font color='red'>Задание 5.</font>
Изучить функцию `pollard_rho(n, s=2, a=1, retries=5, seed=1234, max_steps=None, F=None)` из модуля `sympy.ntheory.residue_ntheory`. Что означают переменные `s, a, retries, seed, max_steps, F`? Какой из изученных выше алгоритмов использует функция `pollard_rho`?

+ `s` - первое значение, подставляемое в функцию `F` (начальная позиция) 
+ `a` - параметр для функции $F =  x^2 + a$; если функция `F` передана в качестве аргумента, то значение `a` не будет использовано (по крайней мере при первой попытке из retries попыток); если retries > 0, то  `a` будет выбрано для следующих попыток случайно. 
+ `retries` - количество попыток для поиска делителя числа n; каждую попытку случайным образом обновляется параметр `a` в функции $F =  x^2 + a$ для генерации последовательности  
+ `seed` - величина, используемая для создания генератора случайных чисел 
+ `max_steps`  если этот аргумент задан, то это максимальное число итераций цикла while; в противном случае цикл не завершится, пока не будет найдет делитель n, либо пока не будет установлено, что алгоритм не выдаст делителя (условие `g == n`) 
+ `F` - функция одного аргумента для генерации последовательности; по умолчанию  $F = x^2 + a$

Функция `pollard_rho` использует алгоритм Черепаха и Заяц.

# 3. $\;\bf(p-1)$-алгоритм Полларда

- Пусть $\;p\;$ -- нетривиальный простой делитель $\;n.$ 

- Пусть известно разложение
$$
  p-1=q_1^{\alpha_1}q_2^{\alpha_2}\dots q_s^{\alpha_s}.
$$

- Согласно Малой теореме Ферма для  $\;a\in\mathbb{N},\;$ $\;\gcd(a,p)=1,\;$ 
$$
  a^{p-1}\equiv 1\mod p.
$$  


- Тогда $$d:=\gcd(a^{p-1}-1,n)\geq p.$$

- Если $\;d<n,\;$ то мы имеем нетривиальный делитель $\;d\;$ числа $\;n.$ 

-  Но нам неизвестны ни число $\;p,\;$ ни его делители $\;q_i.$ 

- Предположим, что нам известны такие числа $\;M,K\in\mathbb{N},\;$ что 
$$
  p<M,\qquad q_i<K
  $$ 
для всех $\;i=1,2,\dots,s.$ 
  
-   Понятно, что, например, при $\;M=K=n\;$ неравенства удовлетворяются, но с вычислительной точки зрения нам такие большие числа $\;M\;$ и $\;K\;$ не подойдут. 
  
-   Через $\;B\;$ обозначим все простые числа, меньшие $\;K,$
  $$
    B:=\{2,3,5,7,\dots,p_i,\dots,p_m\},
  $$
$\;p_i\;$ -- простые, $\;p_i<K.$   

- Так как $\;q_i<K,\;$ то число $\;p-1\;$ имеет разложение
$$
 p-1=p_1^{\alpha_1}p_2^{\alpha_2}\dots p_m^{\alpha_m},
$$
 где $\;\alpha_i\geq0,\;$ но мы не знаем $\;\alpha_i.\;$ 
 
- Так как $\;p<M,\;$ то $\;\alpha_i\leq\log_{p_i}M.$


- Обозначим
   $
     \beta_i:=\left[\log_{p_i}M\right].
   $
   
- Получили число 
   $$
     P:=p_1^{\beta_1}p_2^{\beta_2}\dots p_m^{\beta_m}.
   $$
- Так как $\;\beta_i\geq\alpha_i,\;$ то $\;P\;$ делится на 
$$
 p-1=p_1^{\alpha_1}p_2^{\alpha_2}\dots p_m^{\alpha_m},
$$
поэтому
   $$
     a^P\equiv1\mod p.
   $$
- Следовательно, 
$$
\;d:=\gcd(a^P-1,n)\geq p.
$$ 

## $\bf(p-1)$-алгоритм Полларда
Input: $\;n\;$  - составное число;   $\;M,K\in\mathbb{N}.$  
   
Output: $\;p\;$ - нетривиальный делитель числа $\;n.$
   
1$.$ Находим все простые делители, меньшие $\;K,\;$  
   $
   \qquad\qquad  B:=\{2,3,5,7,\dots,p_i,\dots,p_m\}.
   $
   
2$.$ Выбираем $a:=2.$
   
3$.$ Для $\;i=1,2,\dots,m\;$ выполняем шаги 3.1, 3.2:  
$\qquad$ 3.1. Вычисляем $\;\beta_i:=\left[\log_{p_i}M\right].$  
$\qquad$ 3.2. Вычисляем $\;a:=a^{p_i^{\beta_i}}\mod n.$
   
4$.$ Находим $\;d:=\gcd(a-1,n).\;$
   
5$.$ Если $\;d=1,\;$ то выдаем результат "делитель не найден, возьмите $\;M,K\;$ большими".

6$.$ Если $\;1<d<n,\;$ то выдаем результат $\;d.\;$
   
7$.$ Если $\;d=n,\;$ то выдаем результат "делитель не найден, возьмите $\;M,K\;$ меньшими".

## <font color='red'>Задание 6.</font>
Реализовать $(\rho-1)$-алгоритм Полларда. При помощи $\;\rho$-метода Полларда и $(\rho-1)$-алгоритма Полларда разложить числа `n1` и `n2` на множители. Сравнить фактическое время работы функций. От каких свойств числа $\;n\;$ зависит скорость разложения числа $\;n\;$ на множители при помощи $\;\rho$-метода Полларда? При помощи $(\rho-1)$-алгоритма Полларда?

In [40]:
n1 = 179440801267606692257
n2 = 4389145587418435224785452661044623743

In [20]:
import math

def pollard_rho_minus_one(n: int, M: int, K: int) -> str:
    B = list(primerange(K))
    a = 2
    for i in range(len(B)):
        beta = int(math.log(M,B[i]))
        a = pow(a,pow(B[i],beta), n)
    d = math.gcd(a - 1,n)
    if d == 1:
        return 'делитель не найден, возьмите  𝑀,𝐾  большими'
    if 1 < d < n:
        return d
    if d == n:
        return 'делитель не найден, возьмите  𝑀,𝐾  меньшими'

5429807
33047362690351


Разложим число `n1` на простые множители при помощи $\rho$-метода Полларда.

In [41]:
numbers = list()

start = time.time()

while not isprime(n1):
    a = 2 
    k = TortoiseHare(n1,a,1)
    if isinstance(k,str):
        a += 1
    else:
        n1 //= k
        numbers.append(k)
numbers.append(n1)

stop = time.time()

print(numbers, stop - start)

[5429807, 33047362690351] 0.007999897003173828


Разложим число `n1` на простые множители при помощи $(\rho-1)$-метода Полларда.

In [42]:
n1 = 179440801267606692257
k = 1_000_000

numbers = list()

start = time.time()

while not isprime(n1):
    s = pollard_rho_minus_one(n1,k,k)
    n1 //= s
    numbers.append(s)
numbers.append(n1)

stop = time.time()

print(numbers, stop - start)

[33047362690351, 5429807] 4.658601999282837


Разложим число `n2` на простые множители при помощи $\rho$-метода Полларда.

In [31]:
numbers = list()

start = time.time()

while not isprime(n2):
    a = 2 
    k = TortoiseHare(n2,a,1)
    if isinstance(k,str):
        a += 1
    else:
        n2 //= k
        numbers.append(k)
numbers.append(n2)
        
stop = time.time()
    
print(numbers, stop - start)

[197449926681961, 22229157848653822788263] 49.61047339439392


Разложим число `n2` на простые множители при помощи $(\rho-1)$-метода Полларда.

In [32]:
n2 = 4389145587418435224785452661044623743
k = 1_000_000

numbers = list()

start = time.time()

while not isprime(n2):
    s = pollard_rho_minus_one(n2,k,k)
    n2 //= s
    numbers.append(s)
numbers.append(n2)

stop = time.time()

print(numbers, stop - start)

[22229157848653822788263, 197449926681961] 5.227827787399292


Функция `pollard_pm1` из модуля `sympy.ntheory` имеет следующий вид

In [19]:
import random
from sympy.core.numbers import igcd
from sympy.ntheory.generate import sieve, primerange
import math
def pollard_pm1(n, B=10, a=2, retries=0, seed=1234):
    """
    Use Pollard's p-1 method to try to extract a nontrivial factor
    of ``n``. Either a divisor (perhaps composite) or ``None`` is returned.

    The value of ``a`` is the base that is used in the test gcd(a**M - 1, n).
    The default is 2.  If ``retries`` > 0 then if no factor is found after the
    first attempt, a new ``a`` will be generated randomly (using the ``seed``)
    and the process repeated.

    Note: the value of M is lcm(1..B) = reduce(ilcm, range(2, B + 1)).

    A search is made for factors next to even numbers having a power smoothness
    less than ``B``. Choosing a larger B increases the likelihood of finding a
    larger factor but takes longer. Whether a factor of n is found or not
    depends on ``a`` and the power smoothness of the even number just less than
    the factor p (hence the name p - 1).

    Although some discussion of what constitutes a good ``a`` some
    descriptions are hard to interpret. At the modular.math site referenced
    below it is stated that if gcd(a**M - 1, n) = N then a**M % q**r is 1
    for every prime power divisor of N. But consider the following:

        >>> from sympy.ntheory.factor_ import smoothness_p, pollard_pm1
        >>> n=257*1009
        >>> smoothness_p(n)
        (-1, [(257, (1, 2, 256)), (1009, (1, 7, 16))])

    So we should (and can) find a root with B=16:

        >>> pollard_pm1(n, B=16, a=3)
        1009

    If we attempt to increase B to 256 we find that it doesn't work:

        >>> pollard_pm1(n, B=256)
        >>>

    But if the value of ``a`` is changed we find that only multiples of
    257 work, e.g.:

        >>> pollard_pm1(n, B=256, a=257)
        1009

    Checking different ``a`` values shows that all the ones that didn't
    work had a gcd value not equal to ``n`` but equal to one of the
    factors:

        >>> from sympy.core.numbers import ilcm, igcd
        >>> from sympy import factorint, Pow
        >>> M = 1
        >>> for i in range(2, 256):
        ...     M = ilcm(M, i)
        ...
        >>> set([igcd(pow(a, M, n) - 1, n) for a in range(2, 256) if
        ...      igcd(pow(a, M, n) - 1, n) != n])
        {1009}

    But does aM % d for every divisor of n give 1?

        >>> aM = pow(255, M, n)
        >>> [(d, aM%Pow(*d.args)) for d in factorint(n, visual=True).args]
        [(257**1, 1), (1009**1, 1)]

    No, only one of them. So perhaps the principle is that a root will
    be found for a given value of B provided that:

    1) the power smoothness of the p - 1 value next to the root
       does not exceed B
    2) a**M % p != 1 for any of the divisors of n.

    By trying more than one ``a`` it is possible that one of them
    will yield a factor.

    Examples
    ========

    With the default smoothness bound, this number can't be cracked:

        >>> from sympy.ntheory import pollard_pm1
        >>> pollard_pm1(21477639576571)

    Increasing the smoothness bound helps:

        >>> pollard_pm1(21477639576571, B=2000)
        4410317

    Looking at the smoothness of the factors of this number we find:

        >>> from sympy.ntheory.factor_ import smoothness_p, factorint
        >>> print(smoothness_p(21477639576571, visual=1))
        p**i=4410317**1 has p-1 B=1787, B-pow=1787
        p**i=4869863**1 has p-1 B=2434931, B-pow=2434931

    The B and B-pow are the same for the p - 1 factorizations of the divisors
    because those factorizations had a very large prime factor:

        >>> factorint(4410317 - 1)
        {2: 2, 617: 1, 1787: 1}
        >>> factorint(4869863-1)
        {2: 1, 2434931: 1}

    Note that until B reaches the B-pow value of 1787, the number is not cracked;

        >>> pollard_pm1(21477639576571, B=1786)
        >>> pollard_pm1(21477639576571, B=1787)
        4410317

    The B value has to do with the factors of the number next to the divisor,
    not the divisors themselves. A worst case scenario is that the number next
    to the factor p has a large prime divisisor or is a perfect power. If these
    conditions apply then the power-smoothness will be about p/2 or p. The more
    realistic is that there will be a large prime factor next to p requiring
    a B value on the order of p/2. Although primes may have been searched for
    up to this level, the p/2 is a factor of p - 1, something that we don't
    know. The modular.math reference below states that 15% of numbers in the
    range of 10**15 to 15**15 + 10**4 are 10**6 power smooth so a B of 10**6
    will fail 85% of the time in that range. From 10**8 to 10**8 + 10**3 the
    percentages are nearly reversed...but in that range the simple trial
    division is quite fast.

    References
    ==========

    .. [1] Richard Crandall & Carl Pomerance (2005), "Prime Numbers:
           A Computational Perspective", Springer, 2nd edition, 236-238
    .. [2] http://modular.math.washington.edu/edu/2007/spring/ent/ent-html/node81.html
    .. [3] https://www.cs.toronto.edu/~yuvalf/Factorization.pdf
    """

    n = int(n)
    if n < 4 or B < 3:
        raise ValueError('pollard_pm1 should receive n > 3 and B > 2')
    prng = random.Random(seed + B)

    # computing a**lcm(1,2,3,..B) % n for B > 2
    # it looks weird, but it's right: primes run [2, B]
    # and the answer's not right until the loop is done.
    for i in range(retries + 1):
        aM = a
        for p in sieve.primerange(2, B + 1):
            e = int(math.log(B, p))
            aM = pow(aM, pow(p, e), n)
        g = igcd(aM - 1, n)
        if 1 < g < n:
            return int(g)

        # get a new a:
        # since the exponent, lcm(1..B), is even, if we allow 'a' to be 'n-1'
        # then (n - 1)**even % n will be 1 which will give a g of 0 and 1 will
        # give a zero, too, so we set the range as [2, n-2]. Some references
        # say 'a' should be coprime to n, but either will detect factors.
        a = prng.randint(2, n - 2)

pollard_pm1(121)

11

## <font color='red'>Задание 7*.</font>
Сравнить функцию `pollard_pm1(n, B=10, a=2, retries=0, seed=1234)` с Вашей реализацией $(p-1)$-алгоритма Полларда. 

Пусть `n =  795413349580631436379`. Найдутся ли такие три числа $a_1,a_2,b\in\mathbb{N},$ $\gcd(a_1,n)=\gcd(a_2,n)=1,$ что операция
`pollard_pm1(n, B=b, a=a1)` выдаст нетривиальный делитель числа $n,$ а операция `pollard_pm1(n, B=b, a=a2)` - нет.

Первому решившему +1 балл к экзаменационной оценке.

# 4. Функция `factorint`
Функция `factorint` из модуля `sympy.ntheory`  использует следующие алгоритмы:  
- Метод пробных делений (`_trial`);
- Метод факторизации Ферма;
- $\rho$-метод Полларда (`pollard_rho`);
- $(p-1)$-метод Полларда (`pollard_pm1`);
- Функцию проверки на простоту (`isprime`);
- Функцию проверки на полную степень: $n = p^\alpha$ (`perfect_power`);
- Метод Ленстры факторизации с помощью эллиптических кривых (`_ecm_one_factor`).

Не менее интересными являются и правила поочередного использования данных функций в функции `factorint`.

In [2]:
from sympy.ntheory import randprime, factorint 
b = 25
p1 = randprime(1<<b-1, 1<<b)
p2 = randprime(1<<b-1, 1<<b)
factorint(p1*p2,verbose=True)

Factoring 813190338184339
Trial division with ints [2 ... 32768] and fail_max=600
Check for termination
Trial division with primes [1805 ... 3610]
Pollard's p-1 with smoothness bound 1805 and seed 3610
Pollard's rho with retries 1, max_steps 1805 and seed 3610
Factoring 32698889
Trial division with ints [2 ... 32768] and fail_max=600
Check for termination
Factorization is complete.
Check for termination
Factorization is complete.


{32698889: 1, 24869051: 1}

## <font color='red'>Задание 8.</font>
Что означают переменные функции `factorint(n, limit=None, use_trial=True, use_rho=True, use_pm1=True, use_ecm=True, verbose=False, visual=None, multiple=False)`?

+ `n` - число, которое нужно разложить на простые множители 
+ `limit` функция возвращает словарь, выполнив пробные деления (или использовав $\rho$-метод Полларда или $(p-1)$-метод Полларда) `limit` раз; в этом словаре будут как уже вычисленные простые множители числа `n`, так и составные делители `n`, которые функция "не успела" разложить. 
+ `use_trial` eсли установлено значение False, то в алгоритме не будет использован метод пробных делений  
+ `use_rho`  eсли установлено значение False, то в алгоритме не будет использован $\rho$-метод Полларда 
+ `use_pm1`  eсли установлено значение False, то в алгоритме не будет использован $(p-1)$-метод Полларда 
+ `use_ecm`eсли установлено значение False, то в алгоритме не будет использован Метод Ленстры факторизации с помощью эллиптических кривых 
+ `verbose` eсли установлено значение True, то выводятся этапы вычисления 
+ `visual` - возвращается строковое представление числа `n` в виде канонического разложения на простые множители 
+ `multiple` eсли установлено значение True, то возвращается список, содержащий простые множители с повторениями.